In [1]:
from agential.cog.lats.strategies.qa import LATSQAStrategy 
from agential.cog.lats.strategies.math import LATSMathStrategy 
from agential.cog.lats.strategies.code import LATSCodeStrategy 
from agential.llm.llm import LLM
from agential.cog.fewshots.hotpotqa import (
    HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
)
from agential.cog.fewshots.gsm8k import (
    GSM8K_FEWSHOT_EXAMPLES_REACT
)
from agential.cog.fewshots.humaneval import (
    HUMANEVAL_FEWSHOT_EXAMPLES_REACT
)
from agential.cog.fewshots.mbpp import (
    MBPP_FEWSHOT_EXAMPLES_REACT
)
from agential.cog.lats.prompts import (
    HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT,
    LATS_INSTRUCTION_HOTPOTQA,
    LATS_REFLECT_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_LATS_VALUE, 
    LATS_VALUE_INSTRUCTION_HOTPOTQA,

    GSM8K_FEWSHOT_EXAMPLES_LATS_REFLECT,
    GSM8K_FEWSHOT_EXAMPLES_LATS_VALUE,
    LATS_INSTRUCTION_GSM8K,
    LATS_REFLECT_INSTRUCTION_GSM8K,
    LATS_VALUE_INSTRUCTION_GSM8K,

    MBPP_FEWSHOT_EXAMPLES_LATS_REFLECT,
    LATS_INSTRUCTION_MBPP,
    LATS_REFLECT_INSTRUCTION_MBPP,
    MBPP_FEWSHOT_EXAMPLES_LATS_VALUE,
    LATS_VALUE_INSTRUCTION_MBPP,

    HUMANEVAL_FEWSHOT_EXAMPLES_LATS_REFLECT,
    LATS_INSTRUCTION_HUMANEVAL,
    LATS_REFLECT_INSTRUCTION_HUMANEVAL,
    HUMANEVAL_FEWSHOT_EXAMPLES_LATS_VALUE,
    LATS_VALUE_INSTRUCTION_HUMANEVAL,
)

from agential.cog.lats.functional import get_node_trajectory_qa, get_node_trajectory_math, get_node_trajectory_code


llm = LLM("gpt-3.5-turbo")

In [2]:
from agential.llm.llm import MockLLM

inst = {"task_id": "HumanEval/0", "prompt": "from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    \"\"\" Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    \"\"\"\n", "entry_point": "has_close_elements", "canonical_solution": "    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n", "test": "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n"}
question = inst['prompt']
key = f"{inst['test']}\ncheck({inst['entry_point']})"



examples = HUMANEVAL_FEWSHOT_EXAMPLES_REACT
reflect_examples = HUMANEVAL_FEWSHOT_EXAMPLES_LATS_REFLECT
value_examples = HUMANEVAL_FEWSHOT_EXAMPLES_LATS_VALUE
prompt = LATS_INSTRUCTION_HUMANEVAL
reflect_prompt = LATS_REFLECT_INSTRUCTION_HUMANEVAL
value_prompt = LATS_VALUE_INSTRUCTION_HUMANEVAL
additional_keys = {}
reflect_additional_keys = {}
value_additional_keys = {}

strategy = LATSCodeStrategy(
    llm=llm,
    n_samples=2,
    max_reflections=4,
    depth_limit=3,
    max_unique=5,
    cache_values=True,
)
out = strategy.generate(
    question=question,
    key=key,
    examples=examples,
    reflect_examples=reflect_examples,
    value_examples=value_examples,
    prompt=prompt,
    reflect_prompt=reflect_prompt,
    value_prompt=value_prompt,
    additional_keys=additional_keys,
    reflect_additional_keys=reflect_additional_keys,
    value_additional_keys=value_additional_keys,
    max_iterations=3,
    reset=True,
)

<PROMPT AGENT======================================================================>
Answer a coding question with interleaving Thought, Action, Observation steps. Thought can reason about the current question and plan the retrieval steps, and Action can be three types:
(1) Implement[<insert your code here>], which implements the function to answer the question.
(2) Test[<insert your code here>], which implements assert statement test cases to test the implemented code.
(3) Finish[<insert your answer here>], which returns the code implementation and finishes the task.

Here are some examples:

(END OF EXAMPLES)

```python
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

    pass
```




Thou

In [6]:
out.additional_info

[LATSStepOutput(iteration=0, current_node={'state': LATSReActStepOutput(thought='', action_type='', query='', observation='', answer='', external_tool_info={}), 'visits': 0, 'value': 0, 'depth': 0, 'is_terminal': False, 'reward': 0}, children_nodes=[{'state': LATSReActStepOutput(thought='I need to iterate through the list of numbers and compare each pair to see if the absolute difference is less than the threshold.', action_type='Implement', query='from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False', observation='\n```python\nfrom typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < thr

In [7]:
out.total_tokens

20947

In [9]:
examples = HUMANEVAL_FEWSHOT_EXAMPLES_REACT
reflect_examples = HUMANEVAL_FEWSHOT_EXAMPLES_LATS_REFLECT
value_examples = HUMANEVAL_FEWSHOT_EXAMPLES_LATS_VALUE
prompt = LATS_INSTRUCTION_HUMANEVAL
reflect_prompt = LATS_REFLECT_INSTRUCTION_HUMANEVAL
value_prompt = LATS_VALUE_INSTRUCTION_HUMANEVAL
additional_keys = {}
reflect_additional_keys = {}
value_additional_keys = {}

responses = [
    'I need to iterate through the list of numbers and compare each pair to see if the absolute difference is less than the threshold.\n\nAction 1: Implement\n\n```python\nfrom typing import List\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```\n\nThought 2: I need to test the implemented function with some test cases.\n\nAction 2: Test\n\n```python\nassert has_close_elements([1.0, 2.0, 3.0], 0.5) == False\nassert has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) == True\n```\n\nThought 3: The function should be able to correctly check if any two numbers in the list are closer than the given threshold.\n\nAction 3: Finish\n\n```python\nfrom typing import List\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```',
    'Implement the code to iterate through the list of numbers and compare each pair to check if the absolute difference is less than the threshold.\n\n```python\nfrom typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```\n\nThought 2: Now, I need to test the implemented code with some test cases to verify its correctness.\nAction 2: \n\n```python\ndef test_has_close_elements():\n    assert has_close_elements([1.0, 2.0, 3.0], 0.5) == False\n    assert has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) == True\n\ntest_has_close_elements()\n```\n\nThought 3: The implemented function passed the test cases successfully, so I can finalize and finish this coding task.\nAction 3: \n\n```python\nfrom typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```',
    'To solve this problem, I will iterate through the list of numbers and compare each pair of numbers to see if they are closer to each other than the given threshold.\n\nAction 1: Implement\n\n```python\nfrom typing import List\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```\n\nObservation 1:\nI have implemented the function to iterate through the list and compare each pair of numbers to see if they are closer than the given threshold. The function will return True if any pair is close enough, otherwise False.\n\n\nThought 2:\nNow, I will test the implemented function with some test cases.\n\nAction 2: Test\n\n```python\nassert has_close_elements([1.0, 2.0, 3.0], 0.5) == False\nassert has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) == True\n```\n\nObservation 2:\nThe tests passed successfully, indicating that the function is working as expected.\n\n\nThought 3:\nI have completed the implementation and testing of the function.\n\nAction 3: Finish\n\n```python\nfrom typing import List\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```',
    'Implement \n\n```python\nfrom typing import List\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```\n\nThought 2: Now that I have implemented the function, I need to test it with some test cases to ensure it works as expected.\nAction 2: \n\n```python\nTest \nassert has_close_elements([1.0, 2.0, 3.0], 0.5) == False\nassert has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) == True\n```\n\nThought 3: The function has passed the test cases successfully. I can now finish and answer the question.\nAction 3: \n\nFinish\n\n```python\nfrom typing import List\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```',
    'The trajectory is incomplete, but the implementation correctly iterates through the list of numbers and compares each pair to see if the absolute difference is less than the threshold. However, the trajectory does not include testing the function to verify its correctness, leading to uncertainty about the accuracy of the solution.\n\nCorrectness score: 5',
    'The trajectory correctly identifies the need to compare pairs of numbers in the list to see if they are closer to each other than the given threshold. The implementation iterates through all pairs of numbers and returns True if a pair is found that meets the condition. However, the implementation only considers pairs of numbers in the order they appear in the list and not all possible pairs.\n\nCorrectness Score: 6',
    'I need to test the function implementation with some test cases to verify its correctness.\nAction 2: Test[\n```python\nassert has_close_elements([1.0, 2.0, 3.0], 0.5) == False\nassert has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) == True\n```\n]\nObservation 2: \n```python\nassert has_close_elements([1.0, 2.0, 3.0], 0.5) == False\nassert has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) == True\n```\nExecution Status: \nThought 3: ',
    'Test[\n```python\nassert has_close_elements([1.0, 2.0, 3.0], 0.5) == False\nassert has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) == True\nassert has_close_elements([1.0, 1.3, 1.2, 1.1], 0.1) == True\nassert has_close_elements([5.0, 6.0, 7.0, 8.0, 9.0], 1.0) == False\n```\n]\nObservation 2: All test cases passed successfully. The function is correctly implemented.\nAction 3: Finish[\n```python\nfrom typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```\n]',
    'I should test this code with some sample test cases to verify its correctness.\nAction 2: Test[\n```python\nassert has_close_elements([1.0, 2.0, 3.0], 0.5) == False\nassert has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) == True\n```\n]\nObservation 2: \n```python\nassert has_close_elements([1.0, 2.0, 3.0], 0.5) == False\nassert has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) == True\n```\nExecution Status: All test cases passed.\n\nThought 3: The code is implemented correctly and passed test cases. I will finish this task by submitting the code.\nAction 3: Finish[\n```python\nfrom typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```\n]',
    "Test[\n```python\nfrom typing import List\n\ndef test_has_close_elements():\n    assert has_close_elements([1.0, 2.0, 3.0], 0.5) == False\n    assert has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) == True\n\ntest_has_close_elements()\n```\n]\nObservation 2:\n```python\nfrom typing import List\n\ndef test_has_close_elements():\n    assert has_close_elements([1.0, 2.0, 3.0], 0.5) == False\n    assert has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) == True\n\ntest_has_close_elements()\n```\nTest Result: All test cases passed successfully.\nThought 3: Since the code implementation passed all the given test cases, I can consider this task complete.\nAction 3: Finish[\n```python\nfrom typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```\n]\nObservation 3: The task has been successfully completed with the implementation of the 'has_close_elements' function.",
    'The trajectory is incorrect because it does not correctly compare the absolute difference between the numbers with the threshold value. The function implementation should compare the absolute difference with the threshold in a way that handles all pairs of numbers in the list properly.\n\nCorrectness score: 2',
    'The code implementation correctly iterates through the list of numbers and compares each pair to check if the absolute difference is less than the threshold. \n\nHowever, there is an issue with the implementation where the threshold is not correctly calculated in the condition. When comparing the absolute difference between two numbers, it should be compared against the threshold directly but not comparing it directly without any modification.\n\nCorrectness Score: 4',
    'The function passed all the test cases and seems to be working correctly. I can now return the implementation.\nAction 3: Finish[\n```python\nfrom typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n``` \n]\nObservation 3: \n```python\nfrom typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```',
    'Finish[\n```python\nfrom typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```\n]',
    'The implementation is correct and all test cases passed. I can now finish this task.\nAction 3: Finish[\n```python\nfrom typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```\n]\nObservation 3: \n```python\nfrom typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```\nExecution Status: Done',
    'Finish[\n```python\nfrom typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```\n]',
]
llm = MockLLM("gpt-3.5-turbo", responses=responses)
strategy = LATSCodeStrategy(
    llm=llm,
    n_samples=2,
    max_reflections=4,
    depth_limit=3,
    max_unique=5,
    cache_values=True,
    testing=True
)
inst = {"task_id": "HumanEval/0", "prompt": "from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    \"\"\" Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    \"\"\"\n", "entry_point": "has_close_elements", "canonical_solution": "    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n", "test": "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n"}
question = inst['prompt']
key = f"{inst['test']}\ncheck({inst['entry_point']})"

out = strategy.generate(
    question=question,
    key=key,
    examples=examples,
    reflect_examples=reflect_examples,
    value_examples=value_examples,
    prompt=prompt,
    reflect_prompt=reflect_prompt,
    value_prompt=value_prompt,
    additional_keys=additional_keys,
    reflect_additional_keys=reflect_additional_keys,
    value_additional_keys=value_additional_keys,
    max_iterations=3,
    reset=True,
)

<PROMPT AGENT======================================================================>
Answer a coding question with interleaving Thought, Action, Observation steps. Thought can reason about the current question and plan the retrieval steps, and Action can be three types:
(1) Implement[<insert your code here>], which implements the function to answer the question.
(2) Test[<insert your code here>], which implements assert statement test cases to test the implemented code.
(3) Finish[<insert your answer here>], which returns the code implementation and finishes the task.

Here are some examples:

(END OF EXAMPLES)

```python
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

    pass
```




Thou

In [10]:
out.additional_info

[LATSStepOutput(iteration=0, current_node={'state': LATSReActStepOutput(thought='', action_type='', query='', observation='', answer='', external_tool_info={}), 'visits': 0, 'value': 0, 'depth': 0, 'is_terminal': False, 'reward': 0}, children_nodes=[{'state': LATSReActStepOutput(thought='I need to iterate through the list of numbers and compare each pair to see if the absolute difference is less than the threshold.', action_type='Implement', query='from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False', observation='\n```python\nfrom typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < thr

In [ ]:
from agential.llm.llm import MockLLM


responses = [
    "First, I need to calculate how many eggs Janet has left after eating three for breakfast and baking muffins.\nAction 1: Calculate[\n```python\neggs_laid_per_day = 16\neggs_eaten = 3\neggs_baked_into_muffins = 4933828\neggs_sold = eggs_laid_per_day - eggs_eaten - eggs_baked_into_muffins\n```\n]\nObservation 1: \n```python\neggs_laid_per_day = 16\neggs_eaten = 3\neggs_baked_into_muffins = 4933828\neggs_sold = eggs_laid_per_day - eggs_eaten - eggs_baked_into_muffins\n```\nExecution Status: Done\nOutput: eggs_sold = -4933815\nThought 2: The calculation resulted in a negative number because I subtracted the number of eggs baked into muffins incorrectly.\nAction 2: Calculate[\n```python\neggs_laid_per_day = 16\neggs_eaten = 3\neggs_baked_into_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_eaten\neggs_sold = eggs_remaining - eggs_baked_into_muffins\n```\n]\nObservation 2: \n```python\neggs_laid_per_day = 16\neggs_eaten = 3\neggs_baked_into_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_eaten\neggs_sold = eggs_remaining - eggs_baked_into_muffins\n```\nExecution Status: Done\nOutput: eggs_sold = -4933815\nThought 3: The calculation is still incorrect. I need to properly account for the eggs baked into muffins.\nAction 3: Calculate[\n```python\neggs_laid_per_day = 16\neggs_eaten = 3\neggs_baked_into_muffins = 4933828\neggs_laid = eggs_laid_per_day\neggs_eaten_by_janet = eggs_eaten\neggs_used_for_muffins = eggs_baked_into_muffins\neggs_remaining = eggs_laid - eggs_eaten_by_janet - eggs_used_for_muffins\neggs_sold = eggs_remaining\n```\n]\nObservation 3: \n```python\neggs_laid_per_day = 16\neggs_eaten = 3\neggs_baked_into_muffins = 4933828\neggs_laid = eggs_laid_per_day\neggs_eaten_by_janet = eggs_eaten\neggs_used_for_muffins = eggs_baked_into_muffins\neggs_remaining = eggs_laid - eggs_eaten_by_janet - eggs_used_for_muffins\neggs_sold = eggs_remaining\n```\nExecution Status: Done\nOutput: eggs_sold = 13\nThought 4: Janet sells 13 fresh duck eggs daily at the farmers' market.\nAction 4: Finish[\n```python\nanswer = 13 * 2\n```\n]\nObservation 4: \n```python\nanswer = 13 * 2\n```",
    'Calculate[\n```python\neggs_laid_per_day = 16\neggs_eaten_daily = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_eaten_daily - eggs_used_for_muffins\n```\n]',
    "I need to figure out how much money Janet makes at the farmers' market daily.\nAction 1: Calculate[\n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_baked_into_muffins = 4933828\neggs_sold = eggs_laid_per_day - eggs_consumed_for_breakfast - eggs_baked_into_muffins\nprice_per_egg = 2\ntotal_income_per_day = eggs_sold * price_per_egg\nanswer = total_income_per_day\n```\n]\nObservation 1: \n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_baked_into_muffins = 4933828\neggs_sold = eggs_laid_per_day - eggs_consumed_for_breakfast - eggs_baked_into_muffins\nprice_per_egg = 2\ntotal_income_per_day = eggs_sold * price_per_egg\nanswer = total_income_per_day\n```\nExecution Status: Done\nOutput: answer = 30\nThought 2: Janet makes $30 every day at the farmers' market.\nAction 2: Finish[\n```python\nanswer = 30\n```\n]\nObservation 2: \n```python\nanswer = 30\n```",
    'Calculate[\n```python\neggs_laid_per_day = 16\neggs_eaten_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_sold = eggs_laid_per_day - eggs_eaten_for_breakfast - eggs_used_for_muffins\nearning_per_day = eggs_sold * 2\nanswer = earning_per_day\n```\n]',
    'The trajectory is incorrect because it fails to account for the fact that Janet cannot use more eggs for muffins than the total number of eggs laid per day. Therefore, the calculation of `eggs_remaining` is flawed, leading to a lack of answer. \n\nCorrectness score: 1',
    "The trajectory is incorrect because the calculation for `eggs_sold` subtracts the number of eggs eaten for breakfast and used for muffins from the total number of eggs laid per day, resulting in a negative value. This negative value then affects the calculation for `earning_per_day`, leading to an incorrect answer. The correct approach should be to calculate the total number of eggs sold at the farmers' market, not the total number of eggs remaining after breakfast and muffin making. \n\nCorrectness score: 2",
    "I need to calculate how much money Janet makes by selling the remaining eggs at the farmers' market.\nAction 2: Calculate[\n```python\neggs_remaining = eggs_laid_per_day - eggs_eaten_daily - eggs_used_for_muffins\nprice_per_egg = 2\nmoney_daily = eggs_remaining * price_per_egg\n```\n]\nObservation 2:\n```python\neggs_remaining = eggs_laid_per_day - eggs_eaten_daily - eggs_used_for_muffins\nprice_per_egg = 2\nmoney_daily = eggs_remaining * price_per_egg\n```\nExecution Status: Done\nOutput: answer = -9867617",
    'Calculate[\n```python\nprice_per_egg = 2\nmoney_made_daily = eggs_remaining * price_per_egg\nanswer = money_made_daily\n```\n]',
    "I also need to calculate how many eggs Janet sells at the farmers' market.\nAction 2: Calculate[\n```python\neggs_sold_daily = eggs_remaining\nprice_per_egg = 2\nincome_daily = eggs_sold_daily * price_per_egg\n```\n]\nObservation 2: \n```python\neggs_sold_daily = eggs_remaining\nprice_per_egg = 2\nincome_daily = eggs_sold_daily * price_per_egg\n```\nExecution Status: Done\nOutput: answer = 26\nThought 3: Janet makes $26 every day at the farmers' market.\nAction 3: Finish[\n```python\nanswer = 26\n```\n]\nObservation 3:\n```python\nanswer = 26\n```",
    "Calculate[\n```python\neggs_sold_daily = eggs_remaining\nprice_per_egg = 2\nincome_per_day = eggs_sold_daily * price_per_egg\nanswer = income_per_day\n```\n]\nObservation 2: \n```python\neggs_sold_daily = eggs_remaining\nprice_per_egg = 2\nincome_per_day = eggs_sold_daily * price_per_egg\nanswer = income_per_day\n```\nExecution Status: Done\nOutput: answer = -9867638\nThought 3: The negative value for income doesn't make sense. I need to review the calculations to correct the error.\nAction 3: Calculate[\n```python\neggs_laid_per_day = 16\neggs_eaten_daily = 3\neggs_used_for_muffins = 4933828\neggs_sold_daily = eggs_laid_per_day - eggs_eaten_daily - eggs_used_for_muffins\nprice_per_egg = 2\nincome_per_day = eggs_sold_daily * price_per_egg\nanswer = income_per_day\n```\n]\nObservation 3: \n```python\neggs_laid_per_day = 16\neggs_eaten_daily = 3\neggs_used_for_muffins = 4933828\neggs_sold_daily = eggs_laid_per_day - eggs_eaten_daily - eggs_used_for_muffins\nprice_per_egg = 2\nincome_per_day = eggs_sold_daily * price_per_egg\nanswer = income_per_day\n```\nExecution Status: Done\nOutput: answer = -19707642\nThought 4: The negative value for income still doesn't make sense. I need to reevaluate the calculations to find the error.\nAction 4: Calculate[\n```python\neggs_laid_per_day = 16\neggs_eaten_daily = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_eaten_daily - eggs_used_for_muffins\neggs_sold_daily = eggs_remaining\nprice_per_egg = 2\nincome_per_day = eggs_sold_daily * price_per_egg\nanswer = income_per_day\n```\n]\nObservation 4: \n```python\neggs_laid_per_day = 16\neggs_eaten_daily = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_eaten_daily - eggs_used_for_muffins\neggs_sold_daily = eggs_remaining\nprice_per_egg = 2\nincome_per_day = eggs_sold_daily * price_per_egg\nanswer = income_per_day\n```\nExecution Status: Done\nOutput: answer = -19707760\nThought 5: There seems to be a recurring error. I should double-check my calculations.\nAction 5: Calculate[\n```python\neggs_laid_per_day = 16\neggs_eaten_daily = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_eaten_daily - eggs_used_for_muffins\neggs_sold_daily = eggs_remaining\nprice_per_egg = 2\nincome_per_day = eggs_sold_daily * price_per_egg\nanswer = abs(income_per_day)\n```\n]\nObservation 5: \n```python\neggs_laid_per_day = 16\neggs_eaten_daily = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_eaten_daily - eggs_used_for_muffins\neggs_sold_daily = eggs_remaining\nprice_per_egg = 2\nincome_per_day = eggs_sold_daily * price_per_egg\nanswer = abs(income_per_day)\n```\nExecution Status: Done\nOutput: answer = 19707760\nThought 6: Janet makes $19,707,760 every day at the farmers' market.\nAction 6: Finish[\n```python\nanswer = 19707760\n```\n]\nObservation 6: \n```python\nanswer = 19707760\n```",
    'The trajectory is incorrect because the variable `eggs_remaining` was not defined or calculated before trying to use it to calculate the money made daily. The correct approach would be to ensure that the variable `eggs_remaining` is properly calculated before using it in subsequent calculations. \n\nCorrectness score: 3',
    'The trajectory is incorrect because the variable `eggs_remaining` was not defined before using it to calculate the income per day. The calculation for income per day is also incorrect as it does not consider the eggs eaten for breakfast and used for muffins. The correct approach would be to subtract the eggs eaten for breakfast and used for muffins from the eggs laid per day to find out how many eggs are available for sale, and then multiply this by the price per egg to calculate the income per day.\nCorrectness score: 2',
    "I forgot to define the variable `eggs_remaining` before using it in the calculation. I need to go back and correct that.\nAction 3: Calculate[\n```python\neggs_laid_per_day = 16\neggs_eaten_daily = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_eaten_daily - eggs_used_for_muffins\nprice_per_egg = 2\nmoney_made_daily = eggs_remaining * price_per_egg\nanswer = money_made_daily\n```\n]\nObservation 3: \n```python\neggs_laid_per_day = 16\neggs_eaten_daily = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_eaten_daily - eggs_used_for_muffins\nprice_per_egg = 2\nmoney_made_daily = eggs_remaining * price_per_egg\nanswer = money_made_daily\n```\nExecution Status: Done\nOutput: answer = 26\n\nThought 4: Janet makes $26 every day at the farmers' market.\nAction 4: Finish[\n```python\nanswer = 26\n```\n]\nObservation 4: \n```python\nanswer = 26\n```",
    'Calculate[\n```python\neggs_laid_per_day = 16\neggs_eaten_daily = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_eaten_daily - eggs_used_for_muffins\nprice_per_egg = 2\nmoney_made_daily = eggs_remaining * price_per_egg\nanswer = money_made_daily\n```\n]',
    'I forgot to define eggs_remaining before using it to calculate the money made daily. I need to go back and define eggs_remaining first.\nAction 3: Calculate[\n```python\neggs_remaining = 16 - 3 - 4933828\nprice_per_egg = 2\nmoney_made_daily = eggs_remaining * price_per_egg\nanswer = money_made_daily\n```\n]\nObservation 3: \n```python\neggs_remaining = 16 - 3 - 4933828\nprice_per_egg = 2\nmoney_made_daily = eggs_remaining * price_per_egg\nanswer = money_made_daily\n```\nExecution Status: Done\nOutput: answer = -9867646',
    'Calculate[\n```python\neggs_laid_per_day = 16\neggs_eaten_daily = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_eaten_daily - eggs_used_for_muffins\nprice_per_egg = 2\nmoney_made_daily = eggs_remaining * price_per_egg\nanswer = money_made_daily\n```\n]',
    "The trajectory is incorrect because it miscalculates the `eggs_remaining` variable, resulting in a negative value for the total money made daily at the farmers' market. The process of calculating the remaining eggs after breakfast and baking muffins is correct, but the mistake lies in using that value to compute the daily earnings. The error in the calculation led to an incorrect final answer. \nCorrectness score: 2",
    'The trajectory is incorrect because the calculation of eggs_remaining was done incorrectly, leading to a negative value and an inaccurate result for the money made daily. The correct approach would be to calculate eggs remaining by subtracting the eggs eaten for breakfast and used for muffins from the total eggs laid per day. The incorrect calculation of eggs_remaining caused the subsequent calculations to be wrong as well, resulting in an incorrect final answer. \n\nCorrectness score: 2',
    "The calculation resulted in a negative value, which doesn't make sense. I need to review my calculations to find the error.\nAction 4: Finish[\n```python\nanswer = -9867630\n```\n]\nObservation 4:\n```python\nanswer = -9867630\n```",
    "Calculate[\n```python\neggs_laid_per_day = 16\neggs_eaten_daily = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_eaten_daily - eggs_used_for_muffins\nprice_per_egg = 2\nmoney_made_daily = eggs_remaining * price_per_egg\nanswer = abs(money_made_daily)  # Taking the absolute value to ensure a positive result\n```\n]\nObservation 4: \n```python\neggs_laid_per_day = 16\neggs_eaten_daily = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_eaten_daily - eggs_used_for_muffins\nprice_per_egg = 2\nmoney_made_daily = eggs_remaining * price_per_egg\nanswer = abs(money_made_daily)  # Taking the absolute value to ensure a positive result\n```\nExecution Status: Done\nOutput: answer = 9867630\nThought 5: Janet makes $9,867,630 every day at the farmers' market.\nAction 5: Finish[\n```python\nanswer = 9867630\n```\n]\nObservation 5: \n```python\nanswer = 9867630\n```",
    "The calculation seems to have resulted in a negative value, which doesn't make sense in this context. I need to review the subtraction logic.\nAction 4: Calculate[\n```python\neggs_laid_per_day = 16\neggs_eaten_daily = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_eaten_daily - eggs_used_for_muffins\nprice_per_egg = 2\nif eggs_remaining < 0:\n    eggs_remaining = 0\nmoney_made_daily = eggs_remaining * price_per_egg\nanswer = money_made_daily\n```\n]\nObservation 4: \n```python\neggs_laid_per_day = 16\neggs_eaten_daily = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_eaten_daily - eggs_used_for_muffins\nprice_per_egg = 2\nif eggs_remaining < 0:\n    eggs_remaining = 0\nmoney_made_daily = eggs_remaining * price_per_egg\nanswer = money_made_daily\n```\nExecution Status: Done\nOutput: answer = 32",
    "Calculate[\n```python\neggs_laid_per_day = 16\neggs_eaten_daily = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_eaten_daily - eggs_used_for_muffins\nif eggs_remaining < 0:\n    eggs_remaining = 0\nprice_per_egg = 2\nmoney_made_daily = eggs_remaining * price_per_egg\nanswer = money_made_daily\n```\n]\nObservation 4: \n```python\neggs_laid_per_day = 16\neggs_eaten_daily = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_eaten_daily - eggs_used_for_muffins\nif eggs_remaining < 0:\n    eggs_remaining = 0\nprice_per_egg = 2\nmoney_made_daily = eggs_remaining * price_per_egg\nanswer = money_made_daily\n```\nExecution Status: Done\nOutput: answer = 0\nThought 5: Based on the corrected logic, Janet makes $0 every day at the farmers' market.\nAction 5: Finish[\n```python\nanswer = 0\n```\n]\nObservation 5: \n```python\nanswer = 0\n```",
    "The trajectory is incorrect because it miscalculated the number of eggs remaining after breakfast and baking muffins. This led to a negative result when calculating the amount of money made at the farmers' market. By correcting the calculation and taking the absolute value of the final result, the trajectory eventually arrived at the correct answer. The correctness score is 5, as the trajectory made mistakes in the initial calculations but corrected them eventually.",
    "The trajectory is incorrect because it miscalculated the number of eggs remaining after accounting for breakfast consumption and muffin production. The corrected approach fixed the negative value issue by setting the remaining eggs to zero if the calculated value is negative. However, the calculation logic still requires adjustment to accurately represent the scenario. The trajectory's final result of 0 is incorrect in the context of the problem. \n\nCorrectness score: 3",
    "The result is $9,867,630 that Janet makes every day at the farmers' market.",
    'Finish[\n```python\nanswer = 9867630\n```\n]\nObservation 5: \n```python\nanswer = 9867630\n```',
    "Janet makes $9,867,630 every day at the farmers' market. \nAction 5: Finish[\n```python\nanswer = 9867630\n```\n]\nObservation 5: \n```python\nanswer = 9867630\n```",
    'Finish[\n```python\nanswer = 9867630\n```\n]\nObservation 5: \n```python\nanswer = 9867630\n```',
]

question = "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with 4933828. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"
key = -9867630

llm = MockLLM("gpt-3.5-turbo", responses=responses)
strategy = LATSMathStrategy(
    llm=llm, 
    n_samples=2, 
    max_reflections=4, 
    depth_limit=5,
    max_unique=5,
    cache_values=True,
    testing=True
)
mock_out = strategy.generate(
    question=question,
    key=key,
    examples=GSM8K_FEWSHOT_EXAMPLES_REACT,
    reflect_examples=GSM8K_FEWSHOT_EXAMPLES_LATS_REFLECT,
    value_examples=GSM8K_FEWSHOT_EXAMPLES_LATS_VALUE,
    prompt=LATS_INSTRUCTION_GSM8K,
    reflect_prompt=LATS_REFLECT_INSTRUCTION_GSM8K,
    value_prompt=LATS_VALUE_INSTRUCTION_GSM8K,
    additional_keys={},
    reflect_additional_keys={},
    value_additional_keys={},
    max_iterations=1,
    reset=True,
)

In [ ]:
question = "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with 4933828. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"
key = -9867630

strategy = LATSMathStrategy(
    llm=llm, 
    n_samples=2, 
    max_reflections=4, 
    depth_limit=5,
    max_unique=5,
    cache_values=True,
    testing=True
)
out = strategy.generate(
    question=question,
    key=key,
    examples=GSM8K_FEWSHOT_EXAMPLES_REACT,
    reflect_examples=GSM8K_FEWSHOT_EXAMPLES_LATS_REFLECT,
    value_examples=GSM8K_FEWSHOT_EXAMPLES_LATS_VALUE,
    prompt=LATS_INSTRUCTION_GSM8K,
    reflect_prompt=LATS_REFLECT_INSTRUCTION_GSM8K,
    value_prompt=LATS_VALUE_INSTRUCTION_GSM8K,
    additional_keys={},
    reflect_additional_keys={},
    value_additional_keys={},
    max_iterations=1,
    reset=True,
)

# print(get_node_trajectory_math(out.answer))

In [ ]:
out.answer.to_dict()

In [ ]:
out.additional_info

In [ ]:
for i in [
    "total_completion_cost",
    "total_completion_tokens",
    "total_prompt_cost",
    "total_prompt_tokens",
    "total_tokens",
    "total_cost",
    "total_prompt_time",
    "total_time"
]:
    print(getattr(out, i))

In [ ]:
import importlib
from agential.cog.lats.strategies import code

# Reload the module
importlib.reload(code)

# Now use the updated module
from agential.cog.lats.strategies.code import LATSCodeStrategy


In [ ]:
inst = {"task_id": "HumanEval/0", "prompt": "from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    \"\"\" Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    \"\"\"\n", "entry_point": "has_close_elements", "canonical_solution": "    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n", "test": "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n"}
question = inst['prompt']
key = f"{inst['test']}\ncheck({inst['entry_point']})"

agent = LATSCodeStrategy(
    llm=llm, 
    n_samples=2, 
    max_reflections=4, 
    depth_limit=5,
    max_unique=5,
    cache_values=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=HUMANEVAL_FEWSHOT_EXAMPLES_REACT,
    reflect_examples=HUMANEVAL_FEWSHOT_EXAMPLES_LATS_REFLECT,
    value_examples=HUMANEVAL_FEWSHOT_EXAMPLES_LATS_VALUE,
    prompt=LATS_INSTRUCTION_HUMANEVAL,
    reflect_prompt=LATS_REFLECT_INSTRUCTION_HUMANEVAL,
    value_prompt=LATS_VALUE_INSTRUCTION_HUMANEVAL,            
    additional_keys={},
    reflect_additional_keys={},
    value_additional_keys={},
    max_iterations=1,
    reset=True
)

print(get_node_trajectory_code(out.answer))

In [ ]:
out

In [ ]:
question = "Write a python function to find the first repeated character in a given string."
key = """assert first_repeated_char("abcabc") == "a"
assert first_repeated_char("abc") == None
assert first_repeated_char("123123") == "1\""""

agent = LATSCodeStrategy(
    llm=llm, 
    n_samples=2, 
    max_reflections=4, 
    depth_limit=5,
    max_unique=5,
    cache_values=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=MBPP_FEWSHOT_EXAMPLES_REACT,
    reflect_examples=MBPP_FEWSHOT_EXAMPLES_LATS_REFLECT,
    value_examples=MBPP_FEWSHOT_EXAMPLES_LATS_VALUE,
    prompt=LATS_INSTRUCTION_MBPP,
    reflect_prompt=LATS_REFLECT_INSTRUCTION_MBPP,
    value_prompt=LATS_VALUE_INSTRUCTION_MBPP,            
    additional_keys={"tests": key},
    reflect_additional_keys={"tests": key},
    value_additional_keys={"tests": key},
    max_iterations=1,
    reset=True
)

print(get_node_trajectory_code(out.answer))

In [ ]:
out